In [2]:
import geopandas
from geopandas.tools import geocode

import numpy as np
import pandas as pd
from shapely.geometry import Point, LineString, Polygon
from datetime import datetime

import math
import folium
from folium import Choropleth
from folium.plugins import HeatMap, MarkerCluster

import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
directory_df = pd.read_csv("directory.csv")
directory_df = directory_df[directory_df["Country"] == 'US']
df = directory_df[directory_df["State/Province"] == 'CA']

address_cols = ["Store Name", "Street Address", "City", "State/Province", "Postcode", "Country"]
df = df.fillna('')
df["Address"] = df["Street Address"] + " " + df["City"] + " " + df["State/Province"] 

cols_to_keep = ["Store Number", "Store Name", "Address", "City", "Longitude", "Latitude"]
df = df[cols_to_keep]

df.loc[df["City"]=='Berkeley', 'Longitude'] = ""
df.loc[df["City"]=='Berkeley', 'Latitude'] = ""
#df.drop("City", axis=1, inplace=True)

#df.to_csv("starbucks_locations.csv", index=False)

In [22]:
counties_gdf = geopandas.read_file("geospatial-course-data/CA_Counties/CA_Counties_TIGER2016.shp")
counties_gdf.geometry = counties_gdf.geometry.to_crs(epsg=4326)
counties_gdf.rename(columns={'NAMELSAD': 'name'}, inplace=True)
counties_gdf["GEOID"] = [int(i[1:]) for i in list(counties_gdf["GEOID"])]
counties_gdf['area_sqkm'] = counties_gdf.geometry.to_crs(epsg=3035).area / 10**6
counties_gdf = counties_gdf[['GEOID', 'name', 'area_sqkm', 'geometry']]
counties_gdf.to_file('CA_county_boundaries.shp')

In [10]:
pop_df = pd.read_csv("geospatial-course-data/PEP_2018_PEPANNRES.csv")
pop_df = pop_df[["GEO.id2", "respop72018"]]
pop_df.rename(columns={"GEO.id2": "GEOID", "respop72018": "population"}, inplace=True)

pop_df.to_csv('CA_county_population.csv', index=False)

In [14]:
# note need to clean up this CSV file
# join on income_df["GEO.id2"] and counties_gdf["GEOID"]
income_df = pd.read_csv("geospatial-course-data/ACS_17_5YR_B19001.csv")
income_df = income_df[["GEO.id2", "HD01_VD16", "HD01_VD17"]]
income_df["high_earners"] = income_df["HD01_VD16"] + income_df["HD01_VD17"]
income_df = income_df[["GEO.id2", "high_earners"]]
income_df.rename(columns={"GEO.id2": "GEOID"}, inplace=True)

income_df.to_csv("CA_county_high_earners.csv", index=False)

In [15]:
test = pd.read_csv("CA_county_high_earners.csv")
test.head()

,GEOID,high_earners
0,6001,145696
1,6003,30
2,6005,1220
3,6007,6860
4,6009,2046


In [31]:
#HC01_EST_VC37
df = pd.read_csv("ACS_17_5YR_S0101/ACS_17_5YR_S0101_with_ann.csv")
df = df.iloc[2:]
df = df[["GEO.id2", "HC01_EST_VC37"]]
df.rename(columns={"GEO.id2": "GEOID", "HC01_EST_VC37": "median_age"}, inplace=True)
df.to_csv("CA_county_median_age.csv", index=False)
df.head()

,GEOID,median_age
2,06001,37.3
3,06003,44.9
4,06005,50.6
5,06007,36.9
6,06009,51.6
